In [1]:
!pip install doctran
!pip install langchain


[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
from dotenv import load_dotenv
load_dotenv()

True

In [3]:
from langchain.schema import Document
import json

In [4]:
complaint_email_content = """
Dear Mustang Motors Team,

I'm writing to express my dissatisfaction with a recent car purchase (Mustang GT). Regrettably, the car hasn't lived up to my expectations, and I would like to request a refund.

The car's functionality and performance have not matched the quality advertised through your marketing initiatives and previous customer testimonials. The innovative design and features, as touted by your R&D department, seem inconsistent with the actual vehicle.

Could you please guide me through the refund process? I trust in your ability to resolve this issue promptly and satisfactorily.

Thank you for your attention to this matter.

Best regards,

Max Müller
"""

In [5]:
from langchain.document_transformers import DoctranPropertyExtractor

documents = [Document(page_content=complaint_email_content)]
properties = [
    {
        "name": "category",
        "description": "The type of email this is.",
        "type": "string",
        "enum": ["complaint", "refund_request", "product_feedback", "customer_service", "other"],
        "required": True,
    },
    {
        "name": "mentioned_product",
        "description": "The product mentioned in this email.",
        "type": "string",
        "required": True,
    },
    {
        "name": "issue_description",
        "description": "A brief explanation of the problem encountered with the product.",
        "type": "string",
        "required": True,
    }
]
property_extractor = DoctranPropertyExtractor(properties=properties, openai_api_model="gpt-3.5-turbo")


In [6]:
extracted_document = await property_extractor.atransform_documents(
    documents, properties=properties
)
print(json.dumps(extracted_document[0].metadata, indent=2))

{
  "extracted_properties": {
    "category": "refund_request",
    "mentioned_product": "Mustang GT",
    "issue_description": "Car functionality and performance not as advertised"
  }
}


In [8]:
from langchain.schema import Document
from langchain.document_transformers import DoctranQATransformer


In [9]:
documents = [Document(page_content=complaint_email_content)]
qa_transformer = DoctranQATransformer(openai_api_model="gpt-3.5-turbo")
transformed_document = await qa_transformer.atransform_documents(documents)

In [10]:
print(json.dumps(transformed_document[0].metadata, indent=2))

{
  "questions_and_answers": [
    {
      "question": "What is the reason for your dissatisfaction with the car?",
      "answer": "The car's functionality and performance have not matched the quality advertised through your marketing initiatives and previous customer testimonials. The innovative design and features, as touted by your R&D department, seem inconsistent with the actual vehicle."
    },
    {
      "question": "What specific issues have you encountered with the car?",
      "answer": "The car's functionality and performance have not lived up to my expectations. It lacks the power and acceleration that were promised, and there have been several mechanical issues since I purchased it."
    },
    {
      "question": "Have you contacted our customer service regarding this issue?",
      "answer": "No, I haven't contacted customer service yet."
    },
    {
      "question": "Would you like us to arrange a meeting with our customer service team to discuss your concerns?",
  

In [12]:
from langchain.document_transformers import DoctranTextTranslator

In [13]:
documents = [Document(page_content=complaint_email_content)]
qa_translator = DoctranTextTranslator(language="german", openai_api_model="gpt-3.5-turbo")

In [14]:
translated_document = await qa_translator.atransform_documents(documents)
translated_document[0].page_content

'Sehr geehrtes Mustang Motors Team,\n\nich schreibe Ihnen, um meine Unzufriedenheit mit einem kürzlichen Autokauf (Mustang GT) zum Ausdruck zu bringen. Bedauerlicherweise hat das Auto nicht meinen Erwartungen entsprochen, und ich möchte eine Rückerstattung beantragen.\n\nDie Funktionalität und Leistung des Autos haben nicht der Qualität entsprochen, die durch Ihre Marketinginitiativen und frühere Kundenbewertungen beworben wurde. Das innovative Design und die Funktionen, wie sie von Ihrer Forschungs- und Entwicklungsabteilung angepriesen wurden, scheinen nicht mit dem tatsächlichen Fahrzeug übereinzustimmen.\n\nKönnten Sie mich bitte durch den Rückerstattungsprozess führen? Ich vertraue darauf, dass Sie dieses Problem schnell und zufriedenstellend lösen können.\n\nVielen Dank für Ihre Aufmerksamkeit in dieser Angelegenheit.\n\nMit freundlichen Grüßen,\n\nMax Müller'